In [ ]:
# 使用《复杂》中罗比扫地机器人的例子，进行遗传算法编程
# 场景说明：
# 场地总共是10×10个格子，每个格子可能有三种情况：
#    0. 空
#    1. 罐子
#    2. 墙
# 罗比是一个机器人，他每次都只能看到周围和当前格子的情况
# 也就是其周围“上下左右中”的情况，按照顺序记入一个列表：
#    [0, 0, 0, 0, 0] -> 周围都是空的
#    [2, 0, 0, 1, 0] -> 上面是墙，右面有罐子
# 针对不同情况他的策略是:
#    0:向上走
#    1:向下走
#    2:向左走
#    3:向右走
#    4:随机走
#    5:什么都不做
#    6:捡罐子
# 总共7种策略
#
# 由于需要计算染色体(chromosome)的代表性，所以我们需要确定染色体的长度：
#    罗比能看到范围的情况 3 ^ 5 = 243 种，所以染色体需要243位的长度
#    我们对应染色体列表的chromosome[0]~chromosome[242]分别对应
#    [0,0,0,0,0], [0,0,0,0,0,1], [0,0,0,0,2], ~ ,[2,2,2,2,1], [2,2,2,2,2]
#    染色体的值 chromosome[n] 取值于 0 ~ 6 共 7 种策略。
# 所以需要在总共 7 ^ 243 种策略中寻找最优策略。
# 比如：
#    染色体m为chromosom[m] = "56132....62312" 长度为243, 其意义为
#    [0,0,0,0,0] 采取 5 也就是 什么都不做的策略
#    [0,0,0,0,1] 采取 6 也就是 拣罐子
#    [0,0,0,0,2] 采取 1 也就是 向右走
#    [0,0,0,1,0] 采取 3 也就是 向后走
#    [0,0,0,1,1] 采取 2 也就是 向前走
# 以此类推
# 优化的目的是找到最佳个体，其实就是染色体序列，使其的表现最佳。染色体是基因组合，种群其实就是染色体组合。
#
# 评分策略：
#    有罐子捡起 +10
#    没罐子却捡 -1
#    撞墙 -5

In [1]:
import math, random

In [15]:
class Population:
    def __init__(self, map_size, size, chrom_size, cp, mp, gen_max, max_step=200, g_loop=100):
        # 种群信息
        self.individuals = [] # 个体集合
        self.fitness = [] # 个体适应度集合
        self.selector_probability = [] # 个体选择概率集合
        self.new_individuals = [] # 新一代个体集合
        self.elitists = {
            "chromosome": [],
            "fitness": -15 * max_step,
            "age": 0
        }
        
        self.elitist = {'chromosome': [0, 0], 'fitness': -15*max_step, 'age': 0} # 最佳个体信息
        
        self.size = size # 种群包含个体数量
        self.chromosome_size = chrom_size # 染色体长度
        self.crossover_probability = cp # 个体间染色体交叉概率
        self.mutation_probability = mp # 个体变异概率
        
        self.generation_max = gen_max # 进化最大世代数
        self.age = 0 # 种群当前世代
        self.maxStep = max_step
        self.gen_loop = g_loop
        
        self.map_dim = map_size
        self.map = self.genMap()
        self.actions = 7
        self.sample_space = 243
        self.strategyMX = {} # 用strategyMX来指定染色体中的基因
        self._genStrategy(self.sample_space)
        self.log = []
        
        self.act = {
            0: "up",
            1: "down",
            2: "left",
            3: "right",
            4: "rand",
            5: "none",
            6: "pick"
        }
        self.status = {
            0: "none",
            1: "can",
            2: "wall"
        }
        self.pickpunish = {
            0: -1,
            # 应该不会出现人在墙上
            2: -1,
            1: 10
        }
        
        for i in range(self.size):
            tmpArr = []
            for n in range(self.sample_space):
                tmpArr.append(random.randint(0, self.actions-1))
            self.individuals.append(tmpArr)
            self.fitness.append(0)
            self.selector_probability.append(0)
            self.new_individuals.append([])

    def _genStrategy(self, sNum):
            # act = list(range(self.actions))
            tmp = [0, 0, 0, 0, 0]
            # 3^5=243种枚举策略
            # self.strategyMX[tuple(tmp)] = act
            self.strategyMX[tuple(tmp)] = 0
            # sNum -= 1
            for i in range(1, sNum):
                tmp = self._addEle(tmp)
                self.strategyMX[tuple(tmp)] = i
            return self.strategyMX

    def _addEle(self, tmp):
        m = len(tmp)
        while m >=0 :
            tmp[m-1] += 1
            if tmp[m-1] > 2:
                tmp[m-1] = 0
                m = m - 1
            else:
                return tmp
            
    def _setItem(self, items=[2, 1, 0], probDis=[0, 0.5, 0.5]):
        # 参数是概率分布数组
        p = random.uniform(0, 1)
        cumulative_probability = 0.0
        for item, item_prob in zip(items, probDis):
            cumulative_probability += item_prob
            if p < cumulative_probability:
                return item
    def showMap(self, tmpmap):
        for i in tmpmap:
            print(" ".join(map(str, i)))
    
    def genMap(self):
        tmpmap = []
        # 四边为墙
        upper = [2] * self.map_dim
        bottom = [2] * self.map_dim
        tmpmap.append(upper)
        for row in range(self.map_dim-2):
            tmpmap.append([0] * self.map_dim)
            tmpmap[row + 1][0] = 2
            for col in range(self.map_dim-2):
                tmpmap[row + 1][col+1] = self._setItem()
            tmpmap[row + 1][col+2] = 2
        tmpmap.append(bottom)
        return tmpmap
    
    def lookAround(self, pos, nowMap):
        # 只能看到上下左右中, pos是(x, y), 但要注意其实 x是纵轴，y是横轴
        x, y = pos
        return nowMap[x-1][y], nowMap[x+1][y], nowMap[x][y-1], nowMap[x][y+1], nowMap[x][y]
    
    def score(self, act, pos, nowMap):
        # 这里实际上是完成了解码和评估两步
        # 解决从染色体到具体得分的过程
        pos = list(pos)
#         print(pos, act)
        if act == "rand":
            act = ["up", "down", "left", "right", "pick", "none"][random.randint(0, 5)]
        scoreHash = {
            # “动作”: [x或者y, -1或者1] 用-1乘以这个值完成加减的转换
            "up": [0,1],
            "down": [0, -1],
            "left": [1, 1],
            "right": [1, -1],
            "none": [0, 0],
            #"rand": [random.randint(0, 1), [-1, 1][random.randint(0, 1)]],
            "pick": [0, 0]
        }
        posIDX = scoreHash[act][0]
        posDelta = scoreHash[act][1]
        pos[posIDX] += -1 * posDelta
        if act == "pick":
            sc = self.pickpunish[nowMap[pos[0]][pos[1]]]
            if nowMap[pos[0]][pos[1]] == 1:
#                 print("Picked", nowMap[pos[0]][pos[1]])
                nowMap[pos[0]][pos[1]] = 0
            return sc, tuple(pos)
        elif nowMap[pos[0]][pos[1]] == 2:
            # 墙壁无法走过去，所以要还原位置，其实很简单只要把乘数-1去掉就好
            pos[posIDX] += posDelta
            return -15, tuple(pos)
        return 0, tuple(pos)
    
    def _initPos(self):
#        initPos = [[1,1], [1, 9], [9, 1], [9, 9]]
#         return 1, 1
#        return initPos[random.randint(0, 3)]
        return random.randint(1, self.map_dim-2), random.randint(1, self.map_dim-2)
    
    
    def genLife(self, nowChromo):
        newMap = self.genMap()
#         print("".join(map(str, nowChromo)))
#         self.showMap(newMap)
        pos = self._initPos()
        # print(pos)
        score = 0
        for i in range(self.maxStep):
            envStatus = self.lookAround(pos, newMap)
            genePOS = self.strategyMX[envStatus]
            score_gen, pos = self.score(self.act[nowChromo[genePOS]], pos, newMap)
            score += score_gen
#             print(envStatus, genePOS, nowChromo[genePOS], score)
        return score

    def fitness_func(self):
        for i in range(self.generation_max):
            # wolf probability
            w_p = 0
            weakness = self.generation_max * 0.01
            if i < weakness:
                w_p = (weakness-i)/weakness
            for no, individual in enumerate(self.individuals):
                score = 0
                for n in range(self.gen_loop):
                    score += self.genLife(individual)
                self.fitness[no] = score / self.gen_loop
            if random.random() < w_p:
                self.meetWolf()
            self.evaluate()
            self.getElitist(i)
            # print(self.selector_probability)
            # print(sum(self.fitness)/len(self.fitness))
            self.log.append([i, max(self.fitness), sum(self.fitness)/len(self.fitness), min(self.fitness)])
            self.evolve()
            print(i, ": ".join(map(str, self.log[-1])))
            
        return True
    
    def evaluate(self):
        sp = self.selector_probability
        worst_score = 15 * self.maxStep
        minFIT = abs(min(self.fitness))
#         ft_sum = sum(self.fitness) + worst_score * self.size
        ft_sum = sum(self.fitness) + minFIT * self.size
        
        for i in range(self.size):
#             sp[i] = (self.fitness[i] + worst_score) / ft_sum
            sp[i] = (self.fitness[i] + minFIT) / ft_sum
        
        old = 0
#         for n, p in sorted(enumerate(sp), key= lambda x: x[1]):
#             old += p
#             sp[n] = old 
        for i in range(self.size):
            sp[i] += sp[i-1]
    
    def meetWolf(self):
        minFit = min(self.fitness)
        weakness = [n for n, val in sorted(enumerate(self.fitness), key=lambda x: x[1])][:10]
        for i in weakness:
            self.fitness[i] = minFit
    
    def select(self):
        t = random.random()
        for n, p in enumerate(self.selector_probability):
            if p > t:
                break
        return n
    
    def showCh(self, chromo):
        return "".join(map(str, chromo))
    
    def cross(self, chromo1, chromo2):
        p = random.random()
        cross_pos = random.randint(0, self.sample_space-1)
        new_chromo1 = chromo1[:]
        new_chromo2 = chromo2[:]
        if chromo1 != chromo2 and p < self.crossover_probability:
            # 按照书上的交叉，是随机的点进行交换
            new_chromo1, new_chromo2 = chromo1[: cross_pos], chromo2[: cross_pos]
            new_chromo1.extend(chromo2[cross_pos:])
            new_chromo2.extend(chromo1[cross_pos:])
        return new_chromo1, new_chromo2
    
    def mutate(self, chromo):
        new_chromo = chromo[:]
        p = random.random()
        # print(p, self.mutation_probability)
        if p < self.mutation_probability:
            mutate_idx = random.randint(0, self.sample_space-1)
            mutate_val = list(range(self.actions))[random.randint(0, self.actions-1)]
            # print(mutate_idx, mutate_val, chromo[mutate_idx])
            new_chromo[mutate_idx] = mutate_val
        return new_chromo
    
    def evolve_double(self):
        i = 2
        while True:
            s_chromo1 = self.select()
            s_chromo2 = self.select()
#             print(s_chromo1, s_chromo2)
            (n_chromo1, n_chromo2) = self.cross(
                self.individuals[s_chromo1],
                self.individuals[s_chromo2])
            self.new_individuals[i] = self.mutate(n_chromo1)
            self.new_individuals[i+1] = self.mutate(n_chromo2)
            i += 2
            if i >= self.size:
                break
        self.new_individuals[0] = self.elitists["chromosome"][0][:]
        self.new_individuals[1] = self.elitists["chromosome"][1][:]
        for i in range(self.size):
            self.individuals[i] = self.new_individuals[i][:]
            
    def evolve(self):
        i = 1
        self.new_individuals[0] = self.elitists["chromosome"][:]
        while True:
            s_chromo1 = self.select()
            s_chromo2 = self.select()
            (n_chromo1, n_chromo2) = self.cross(
            self.individuals[s_chromo1],
            self.individuals[s_chromo2])
#             print(s_chromo1, self.fitness[s_chromo1], self.selector_probability[s_chromo1])
#             print(s_chromo2, self.fitness[s_chromo2], self.selector_probability[s_chromo2])
            if random.randint(0, 1) == 0:
                self.new_individuals[i] = self.mutate(n_chromo1)
            else:
                self.new_individuals[i] = self.mutate(n_chromo2)
            
            i += 1
            if i >= self.size:
                break
        for i in range(self.size):
            self.individuals[i] = self.new_individuals[i][:]
    
    def getElitist(self, age):
        bestIndividual = [[idx, fit] for idx, fit in sorted(
            enumerate(self.fitness), key=lambda x: x[1], reverse=True
        )][0]
        if self.elitists["fitness"] < self.fitness[bestIndividual[0]]:
            self.elitists["chromosome"] = []
            self.elitists["age"] = age
            self.elitists["chromosome"].extend(self.individuals[bestIndividual[0]])
            self.elitists["fitness"] = self.fitness[bestIndividual[0]]


In [18]:
m = Population(10, 200, 243, 0.90, 0.2, 2000)
# len(m.individuals)
# n = m.genMap()
# for i in m.map:
#     print("".join(map(str, i)))
# m.lookAround((1,3))

In [ ]:
m.fitness_func()

0 0: -128.27: -584.5571000000002: -1038.07
1 1: -148.64: -524.8088999999999: -946.62
2 2: -104.7: -493.14359999999976: -893.71
3 3: -114.3: -485.9048: -1069.44
4 4: -89.67: -438.77994999999964: -803.18
5 5: -77.18: -398.0283: -859.72
6 6: -29.79: -383.77004999999986: -799.64
7 7: -79.95: -341.05940000000004: -678.29
8 8: -77.27: -302.38804999999974: -647.13
9 9: -28.88: -257.64865000000003: -668.42
10 10: -29.12: -243.86020000000008: -590.37
11 11: -24.85: -219.22629999999998: -471.55
12 12: -25.87: -189.85915: -506.66
13 13: -21.26: -189.61279999999996: -472.84
14 14: -17.5: -158.32975: -471.25
15 15: -23.61: -153.58405: -355.36
16 16: -23.81: -140.88559999999998: -405.5
17 17: -10.95: -130.34394999999998: -341.72
18 18: -7.96: -120.22620000000005: -318.56
19 19: -9.32: -108.52165000000005: -390.81
20 20: -0.92: -105.84740000000004: -463.99
21 21: -16.07: -93.9419: -351.57
22 22: -11.56: -102.16550000000007: -336.1
23 23: -12.76: -95.45450000000001: -261.16
24 24: -10.03: -89.95419999

In [18]:
a.append(m.log)

In [20]:
a[1]

[[0, -169.505, -577.9057249999997, -1220.66],
 [1, -180.575, -557.1205499999998, -1101.01],
 [2, -240.19, -520.3720500000003, -955.93],
 [3, -148.25, -498.30807500000014, -883.46],
 [4, -123.565, -471.9982500000003, -832.91],
 [5, -130.175, -416.3780000000001, -774.315],
 [6, -118.94, -379.398575, -780.145],
 [7, -101.95, -340.5626999999999, -718.005],
 [8, -94.315, -324.28779999999995, -678.665],
 [9, -86.485, -287.003475, -660.35],
 [10, -62.925, -261.980375, -579.74],
 [11, -68.04, -244.94049999999993, -583.015],
 [12, -67.355, -226.6496249999999, -475.665],
 [13, -68.85, -210.36114999999987, -521.28],
 [14, -47.98, -191.31215, -530.405],
 [15, -49.805, -190.10945000000007, -434.085],
 [16, -44.4, -175.16435, -390.255],
 [17, -43.11, -160.599425, -355.15],
 [18, -32.885, -155.60407500000008, -326.385],
 [19, -37.735, -140.54685000000012, -306.27],
 [20, -26.925, -133.17657500000004, -301.84],
 [21, -24.9, -129.9301, -325.655],
 [22, -22.795, -122.46994999999998, -287.325],
 [23, -16

In [133]:
m.lookAround((6, 3)), m.strategyMX[m.lookAround((6, 3))]

((0, 1, 0, 0, 1), [0, 1, 2, 3, 4, 5, 6])

In [193]:
m.initPos()

AttributeError: 'Population' object has no attribute 'initPos'